In [1]:
from datetime import datetime, timedelta
import netCDF4
import numpy as np
import bisect

# Overview of flexible data-loading functionality

1) Below you'll find the function I currently use to subset the data.
All this would subsetting in space & time and more (like averaging currents over time instead of the current temporal slides) should be done via the C3 platform by dynamically loading the hindcast data from the server and/or the 
archive of forecast files. And then just return the right current matrices and grids back for use in the simulator.

2) A sketch of how the function may look like in the end
Of course we can start a lot simpler without any of the averaging/interpolation yet =)

Thanks a lot for looking into this for us, that helps a lot!

# 1) Current implementation of subsetting from nc file example

In [2]:
1.5*24*5*3600/111120

5.831533477321814

In [ ]:
1x1

In [2]:
def get_current_data_subset(nc_file, x_0, x_T, deg_around_x0_xT_box, fixed_time=None,
                            temporal_stride=1, temp_horizon_in_h=None):
    """ Function to read a subset of the nc_file current data bounded by a box spanned by the x_0 and x_T points.
    Inputs:
        nc_file                 full path to nc file
        x_0                     [lat, lon, charge, timestamp in POSIX]
        x_T                     [lon, lat] goal locations
        deg_around_x0_xT_box    float, buffer around the box in degrees
        fixed_time              if None returns time-varying currents, 
                                otherwise datetime object of the fixed time -> returns ocean current grid at or before time
                                the time of x_0 is then ignored
        temporal_stride         int, if a stride of the temporal values is used (every temporal_stride hours)
        temp_horizon            if None: all available time of the file will be provided
                                otherwise float, maximum temp_horizon to look ahead of x_0 time in hours
                                
    Outputs:
        grids_dict              dict containing x_grid, y_grid, t_grid, fixed_time_idx
        u_data                  [T, Y, X] matrix of the ocean currents in x direction in m/s
        v_data                  [T, Y, X] matrix of the ocean currents in y direction in m/s
        
    """
    
    f = netCDF4.Dataset(nc_file)

    # extract positiond & start_time for the indexing
    x_0_pos = x_0[:2]
    x_0_posix_time = x_0[3]
    x_T = x_T[:2]

    # Step 1: get the grids
    xgrid = f.variables['lon'][:]
    ygrid = f.variables['lat'][:]
    t_grid = f.variables['time'][:] # not this is in hours from HYCOM data!
    
    # this is needed because the time origin in hindcast and forecase nc files is different. Very handcrafted.
    try:
        time_origin = datetime.strptime(f.variables['time'].__dict__['time_origin'] + ' +0000',
                                        '%Y-%m-%d %H:%M:%S %z')
    except:
        time_origin = datetime.strptime(f.variables['time'].__dict__['units'] + ' +0000',
                                                 'hours since %Y-%m-%d %H:%M:%S.000 UTC %z')

    # Step 2: find the sub-setting
    # find the lat & lon sub-set bounds
    lon_bnds = [min(x_0_pos[0], x_T[0]) - deg_around_x0_xT_box, max(x_0_pos[0], x_T[0]) + deg_around_x0_xT_box]
    lat_bnds = [min(x_0_pos[1], x_T[1]) - deg_around_x0_xT_box, max(x_0_pos[1], x_T[1]) + deg_around_x0_xT_box]

    # get the respective indices from the grids
    ygrid_inds = np.where((ygrid > lat_bnds[0]) & (ygrid < lat_bnds[1]))[0]
    xgrid_inds = np.where((xgrid > lon_bnds[0]) & (xgrid < lon_bnds[1]))[0]

    # for time indexing transform to POSIX time
    abs_t_grid = [(time_origin + timedelta(hours=X)).timestamp() for X in t_grid.data]
    
    # get the idx of the value left of the demanded time (for interpolation function)
    t_start_idx = bisect.bisect_right(abs_t_grid, x_0_posix_time) - 1
    if t_start_idx == len(abs_t_grid) - 1 or t_start_idx == -1:
        raise ValueError("Requested subset time is outside of the nc4 file.")

    # get the max time if provided as input
    if temp_horizon_in_h is None:   # all data provided
        t_end_idx = len(abs_t_grid)-1
    else:
        t_end_idx = bisect.bisect_right(abs_t_grid, x_0_posix_time + temp_horizon_in_h*3600.)
        if t_end_idx == len(abs_t_grid):
            raise ValueError("nc4 file does not contain requested temporal horizon.")

    # fixed time logic if necessary
    if fixed_time is None:
        slice_for_time_dim = np.s_[t_start_idx:(t_end_idx+1):temporal_stride]
        fixed_time_idx = None
    else:
        fixed_time_idx = bisect.bisect_right(abs_t_grid, fixed_time.timestamp()) - 1
        slice_for_time_dim = np.s_[fixed_time_idx]

    # Step 2: extract data
    # raw water_u is [tdim, zdim, ydim, xdim]
    if len(f.variables['water_u'].shape) == 4:  # if there is a depth dimension in the dataset
        u_data = f.variables['water_u'][slice_for_time_dim, 0, ygrid_inds, xgrid_inds]
        v_data = f.variables['water_v'][slice_for_time_dim, 0, ygrid_inds, xgrid_inds]
    # raw water_u is [tdim, ydim, xdim]
    elif len(f.variables['water_u'].shape) == 3:  # if there is no depth dimension in the dataset
        u_data = f.variables['water_u'][slice_for_time_dim, ygrid_inds, xgrid_inds]
        v_data = f.variables['water_v'][slice_for_time_dim, ygrid_inds, xgrid_inds]
    else:
        raise ValueError("Current data in nc file has neither 3 nor 4 dimensions. Check file.")

    # create dict to output
    grids_dict = {'x_grid': xgrid[xgrid_inds], 'y_grid': ygrid[ygrid_inds],
                  't_grid': abs_t_grid[slice_for_time_dim], 'fixed_time_idx': fixed_time_idx}

    # log what data has been subsetted
    if fixed_time is None:
        print("Subsetted data from {start} to {end} in {n_steps} time steps of {time:.2f} hour(s) resolution".format(
            start=datetime.utcfromtimestamp(grids_dict['t_grid'][0]).strftime('%Y-%m-%d %H:%M:%S UTC'),
            end=datetime.utcfromtimestamp(grids_dict['t_grid'][-1]).strftime('%Y-%m-%d %H:%M:%S UTC'),
            n_steps=len(grids_dict['t_grid']), time=(grids_dict['t_grid'][1] - grids_dict['t_grid'][0])/3600.))
    else:
        print("Subsetted data to fixed time at: {time}".format(
            time=datetime.utcfromtimestamp(grids_dict['t_grid'][0]).strftime('%Y-%m-%d %H:%M:%S UTC')))

    #TODO: we replace the masked array with fill value 0 because otherwise interpolation doesn't work.
    # Though that means we cannot anymore detect if we're on land or not (need a way to do that/detect stranding)
    # not sure yet if we'll do it in the simulator or where.
    # return grids_dict, u_data.filled(fill_value=0.), v_data.filled(fill_value=0.)
    return grids_dict, u_data, v_data

In [3]:
# example execution

# settings
hindcast_file = '2021_06_1-05_hourly.nc4'
x_0 = [-88.0, 25.0, 1, 1622549410.0]  # lon, lat, battery, posix_time
x_T = [-88.0, 26.3]
deg_around_x0_xT_box = 0.5
fixed_time = None
temporal_stride = 1

# function call
grids_dict, u_data, v_data = get_current_data_subset(hindcast_file,
                                                     x_0, x_T,
                                                     deg_around_x0_xT_box,
                                                     fixed_time,
                                                     temporal_stride)

FileNotFoundError: [Errno 2] No such file or directory: b'2021_06_1-05_hourly.nc4'

In [ ]:
c3Grid(SurfaceHindcastData.fetch({"include": "id", "filter": 'parent==HNDCST_SRFC_1-0'}))

In [10]:
import numpy as np
# Step 1: Get the id's of the subset of LatLonPairs via a DB Query
objs_list = c3.HycomLatLongPair.fetch(
spec={"include": "id, lat, lon", "filter": "lat > 18 && lat < 19 && lon > -98 && lon < -97", "order": "j,i"}).objs


ids_list = [obj.id for obj in objs_list]
lat_list = [obj.lat for obj in objs_list]
lon_list = [obj.lon for obj in objs_list]

# create the x, y grids
lat_grid = np.unique(np.array(lat_list))
lon_grid = np.unique(np.array(lon_list))
# ids_list

print(lat_grid)


[18.12000084 18.15999985 18.20000076 18.23999977 18.28000069 18.31999969
 18.36000061 18.39999962 18.44000053 18.47999954 18.52000046 18.55999947
 18.60000038 18.63999939 18.68000031 18.71999931 18.76000023 18.79999924
 18.84000015 18.87999916 18.92000008 18.95999908]


# 2) Sketch of how C3 function interface might look like

In [171]:
def get_current_data_subset_from_c3( 
    t_interval, #temp_res_in_h,
    lat_interval, #lat_res_in_deg,
    lon_interval, #lon_res_in_deg,
    #depth_interval_to_avg_over
):
    # scipy.interpolate.interp1d
    
    """ Function to get a subset of current data via the C3 data integration.
    
    Inputs:
        t_interval              if time-varying: [t_0, t_T] in POSIX time
                                where t_0 and t_T are the start and end timestamps respectively
                                if fixed_time:   [fixed_timestamp] in POSIX
        temp_res_in_h           which temporal resolution the time-axis should have
                                e.g. if temp_res_in_h = 1, t_grid = [t_0, t_0 + 3600s, ... t_T]
                                if temp_res_in_h = 5,      t_grid = [t_0, t_0 + 5*3600s, ... t_T]
                                if temp_res_in_h = 0.5,      t_grid = [t_0, t_0 + 1800s, ... t_T]
                                => so either averaging or interpolation needs to be done in the backend
        lat_interval            [y_lower, y_upper] in degrees
        lat_res_in_deg          which spatial resolution in y direction in degrees
                                e.g. if lat_res_in_deg = 1, y_grid = [y_lower, y_lower + 1, ... y_upper]
                                 => so either averaging or interpolation needs to be done in the backend
        lon_interval            [x_lower, x_upper] in degrees
        lon_res_in_deg          which spatial resolution in x direction in degrees
                                e.g. if lon_res_in_deg = 1, x_grid = [x_lower, x_lower + 1, ... x_upper]
                                 => so either averaging or interpolation needs to be done in the backend
        depth_interval_to_avg_over
                                Interval to average over the current dimension in meters
                                e.g. [0, 10] then the currents are averaged over the depth 0-10m.
                                
    Outputs:
        grids_dict              dict containing x_grid, y_grid, t_grid
        u_data                  [T, Y, X] matrix of the ocean currents in x direction in m/s
        v_data                  [T, Y, X] matrix of the ocean currents in y direction in m/s
    """
    
    # some C3 magic =)
    
    grids_dict = {}
    #Getting time and formatting 
    t_0 = t_interval[0]
    t_T = t_interval[1]
    t_grid = [t_0+i*3600 for i in range(num_interval)]
    if t_T%3600!=0:
        t_grid.append(t_T)
    start_time = datetime.utcfromtimestamp(t_0).strftime("%Y-%m-%d")
    end_time = datetime.utcfromtimestamp(t_T).strftime("%Y-%m-%d") 

    #Getting correct range of nc files from database
    filter_string = 'start>=' + '"'+ start_time + '"' + ' && end<=' + '"' + end_time + "T23:00:00.000" + '"'
    '''===Note===
    I checked the fetch results and found out that the ordering was preserved. Since original ordering was based
    on date, the dates were preserved. If the files in the db themselves are not sorted, we can apply an additional 
    order keyword. For now we don't need it because that might cause extra runtime'''
    objs_list = c3.HindcastFile.fetch({'filter':filter_string}).objs
    ids_list = [obj.id for obj in objs_list]
    
    #Get the first file and use it to extract grid information for us
    # Step 1: Getting the grids
    first_file = c3.HindcastFile.get(ids_list[0])
    first_f = c3.HycomUtil.nc_open(first_file.file.url) 
    xgrid = first_f.variables['lon'][:]
    ygrid = first_f.variables['lat'][:]
    #Get the respective indices from the grids (Same for every file)
    ygrid_inds = np.where((ygrid >= lat_interval[0]) & (ygrid <= lat_interval[1]))[0]
    xgrid_inds = np.where((xgrid >= lon_interval[0]) & (xgrid <= lon_interval[1]))[0]
    print(ygrid.shape)
    print(ygrid_inds)
    
    #Step 2: Getting data from files 
    u_data, v_data = np.array([]), np.array([])
    for i, file_id in enumerate(ids_list):
        file = c3.HindcastFile.get(file_id)
        f = c3.HycomUtil.nc_open(file.file.url)    #water_u dimension: (time,depth,lat,lon)
        start_hr = 0
        end_hr = 24

        #Case 1: file is first -- get data from the file from the hour t_0 is in 
        if i == 0:
            file_t_0 = time.mktime(file.start.timetuple())
            start_hr = int((t_0 - file_t_0)/3600)
        #Case 2: file is last -- get data from file until the hour t_T is in 
        if i == len(ids_list)-1:
            file_t_0 = time.mktime(file.start.timetuple())
            end_hr = math.ceil((t_T-file_t_0)/3600)+1
                 
        #extract data
        curr_u_data = f.variables['water_u'][:][start_hr:end_hr, 0, ygrid_inds, xgrid_inds]
        curr_v_data = f.variables['water_v'][:][start_hr:end_hr, 0, ygrid_inds, xgrid_inds]
        if i == 0:     #if it's the first file, use it to initialize the np array that we are going to append to 
            u_data = np.array(curr_u_data)
            v_data = np.array(curr_v_data)
        else:
            u_data = np.vstack((u_data, curr_u_data))
            v_data = np.vstack((v_data, curr_v_data))

    #Construct grid for output:
    grids_dict = {'x_grid': xgrid[xgrid_inds], 'y_grid': ygrid[ygrid_inds],
                  't_grid': t_grid}
    #Getting data
    return grids_dict, u_data, v_data

In [172]:
grids_dict, u_data, v_data = get_current_data_subset_from_c3([1630540800.0,1630544400.0],[18,19],[-98,-97])

(346,)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22]


IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (23,) (26,) 

(2,)

In [116]:
#time, depth, lat, lon
lon_interval = [-98,-97]
lat_interval = [18,19]
deg_around_x0_xT_box = 0.5
file = c3.HindcastFile.get('9438b440-e200-4764-b6fe-ee6a278aff55/GOMu0.04-expt_90.1m000-2021-2021-09-04T00:00:00Z-2021-09-04T23:00:00Z.nc')
f = c3.HycomUtil.nc_open(file.file.url)
xgrid = f.variables['lon'][:]
ygrid = f.variables['lat'][:]
lon_bnds = [min(lon_interval[0], lon_interval[1]) - deg_around_x0_xT_box, max(lon_interval[0], lon_interval[1]) + deg_around_x0_xT_box]
lat_bnds = [min(lat_interval[0], lat_interval[1]) - deg_around_x0_xT_box, max(lat_interval[0], lat_interval[1]) + deg_around_x0_xT_box]
# get the respective indices from the grids
ygrid_inds = np.where((ygrid > lat_bnds[0]) & (ygrid < lat_bnds[1]))[0]
xgrid_inds = np.where((xgrid > lon_bnds[0]) & (xgrid < lon_bnds[1]))[0]

In [159]:
f.variables['water_u'][:][0:24,0,[0,1],[0,1,2]].shape

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (1,2) (1,3) 

In [156]:
np.array([0,1]).shape

(2,)

In [40]:
ids = c3.HindcastFile.fetch({'filter':'start>="2021-09-02" && end<="2021-09-06"'}).objs
for x in ids:
    print(x.id)

9438b440-e200-4764-b6fe-ee6a278aff55/GOMu0.04-expt_90.1m000-2021-2021-09-02T00:00:00Z-2021-09-02T23:00:00Z.nc
9438b440-e200-4764-b6fe-ee6a278aff55/GOMu0.04-expt_90.1m000-2021-2021-09-03T00:00:00Z-2021-09-03T23:00:00Z.nc
9438b440-e200-4764-b6fe-ee6a278aff55/GOMu0.04-expt_90.1m000-2021-2021-09-04T00:00:00Z-2021-09-04T23:00:00Z.nc
9438b440-e200-4764-b6fe-ee6a278aff55/GOMu0.04-expt_90.1m000-2021-2021-09-05T00:00:00Z-2021-09-05T23:00:00Z.nc


In [41]:
f.variables["lat"][:]

masked_array(data=[18.12000084, 18.15999985, 18.20000076, 18.23999977,
                   18.28000069, 18.31999969, 18.36000061, 18.39999962,
                   18.44000053, 18.47999954, 18.52000046, 18.55999947,
                   18.60000038, 18.63999939, 18.68000031, 18.71999931,
                   18.76000023, 18.79999924, 18.84000015, 18.87999916,
                   18.92000008, 18.95999908, 19.        , 19.04000092,
                   19.07999992, 19.12000084, 19.15999985, 19.20000076,
                   19.23999977, 19.28000069, 19.31999969, 19.36000061,
                   19.39999962, 19.44000053, 19.47999954, 19.52000046,
                   19.55999947, 19.60000038, 19.63999939, 19.68000031,
                   19.71999931, 19.76000023, 19.79999924, 19.84000015,
                   19.87999916, 19.92000008, 19.95999908, 20.        ,
                   20.04000092, 20.07999992, 20.12000084, 20.15999985,
                   20.20000076, 20.23999977, 20.28000069, 20.31999969,
      

In [22]:
t_grid = f.variables['time'][:]
time_origin = datetime.strptime(f.variables['time'].__dict__['time_origin'] + ' +0000','%Y-%m-%d %H:%M:%S %z')
abs_t_grid = [(time_origin + timedelta(hours=X)).timestamp() for X in t_grid.data]
t_0 = 1630540800.0
t_T = 1630567000.0
num_interval = int((t_T - t_0)/(3600*1)+1)
t_grid = [t_0 + i*1*3600 for i in range(num_interval)]

In [101]:
    # create dict to output
    grids_dict = {'x_grid': xgrid[xgrid_inds], 'y_grid': ygrid[ygrid_inds],
                  't_grid': abs_t_grid[slice_for_time_dim], 'fixed_time_idx': fixed_time_idx}

    # log what data has been subsetted
    if fixed_time is None:
        print("Subsetted data from {start} to {end} in {n_steps} time steps of {time:.2f} hour(s) resolution".format(
            start=datetime.utcfromtimestamp(grids_dict['t_grid'][0]).strftime('%Y-%m-%d %H:%M:%S UTC'),
            end=datetime.utcfromtimestamp(grids_dict['t_grid'][-1]).strftime('%Y-%m-%d %H:%M:%S UTC'),
            n_steps=len(grids_dict['t_grid']), time=(grids_dict['t_grid'][1] - grids_dict['t_grid'][0])/3600.))
    else:
        print("Subsetted data to fixed time at: {time}".format(
            time=datetime.utcfromtimestamp(grids_dict['t_grid'][0]).strftime('%Y-%m-%d %H:%M:%S UTC')))


[1630540800.0,
 1630544400.0,
 1630548000.0,
 1630551600.0,
 1630555200.0,
 1630558800.0,
 1630562400.0,
 1630566000.0,
 1630569600.0,
 1630573200.0,
 1630576800.0,
 1630580400.0,
 1630584000.0,
 1630587600.0,
 1630591200.0,
 1630594800.0,
 1630598400.0,
 1630602000.0,
 1630605600.0,
 1630609200.0,
 1630612800.0,
 1630616400.0,
 1630620000.0,
 1630623600.0]

(1, 1, 1)

In [ ]:
    #Finding t_grid according to temp_res_in_h
    #num_interval = (t_T - t_0)/(3600*temp_res_in_h)+1
    #t_grid = [t_0+i*temp_res_in_h*3600 for i in range(num_interval)]
    #if t_T%temp_res_in_h!=0:
        #t_grid.append(t_T)
    